In [20]:
library(dplyr)
library(tidyr)
library(tibble)
library(ggplot2)
library(stringr)
library(ggrepel)
library(ggh4x)
library(grid)
library("cowplot")
library(patchwork)
is_dark_color <- function(hex) {
  rgb <- col2rgb(hex)
  
  luminance <- 0.299 * rgb[1,] + 0.587 * rgb[2,] + 0.114 * rgb[3,]
  if(luminance < 128){
  return('white')
  }else{return('black')}
}

In [21]:
color_annotation<-read.csv('../Color_hex_codes/Cell_Type_Color_Annotation_and_Order.csv')
color_vector <- setNames(color_annotation$color, color_annotation$label)

In [22]:
df<-read.csv('/home//jupyter/BRI_Figures_Final_V2/Figure4/02_Freq_Changes_Y1D0/Freq_OverTime_CMVpos_Supp.csv')

In [23]:
df <- df %>% dplyr::mutate(`Age Group` = case_when(
  cohort.cohortGuid == "BR1" ~ "Young",
  cohort.cohortGuid == "BR2" ~ "Older",
  TRUE ~ NA_character_
))
df$`Age Group`<-factor(df$`Age Group`,levels=c("Young","Older"))

In [24]:
options(repr.plot.width = 50/25.4, repr.plot.height = 70/25.4)

p <- ggplot(df, 
            aes(sample.daysSinceFirstVisit, AIFI_L3_clr,color=`Age Group`)) +
    geom_point(size = 0.75,  stroke = 0) +
    geom_smooth(method="lm",se=FALSE, linewidth=0.2) + 
scale_color_manual(values = c("Older"='#bf812d',  "Young"='#35978f'))+
    theme_bw(base_size = 6) +
    theme(axis.line = element_line(colour = "black", linewidth = 0.1),
          legend.position = "top",
          legend.title = element_blank(),
          panel.grid.major = element_blank(),
          panel.grid.minor = element_blank(),
          panel.border = element_blank(),
          panel.background = element_blank(),
          strip.background = element_rect(linewidth = 0),
          axis.title.x = element_text(margin = margin(t = 2))) +
    facet_wrap2(~AIFI_L3, axes = "all", remove_labels = "all", ncol=1, scale='free_y')  + theme(
    panel.spacing = unit(-0.4,'lines')
  )


In [34]:
celltype_order<-c("KLRF1+ effector Vd1 gdT",
                    'GZMK+ CD27+ EM CD8 T cell')

In [35]:
plot_list<-list()
for (selected_celltype in celltype_order){
df_sub <- df %>% filter(AIFI_L3 == selected_celltype)

p1 <- ggplot(df_sub, aes(sample.daysSinceFirstVisit, AIFI_L3_clr, color = `Age Group`)) +
geom_line(aes(group = interaction(`Age Group`, subject.subjectGuid)), 
              linewidth=0.1, 
              alpha=0.4,
              linetype =1)+
  ylab('') +
  theme_bw(base_size = 6) +
  theme(
    axis.line = element_line(colour = "black"),
    panel.grid.major = element_blank(),
    panel.grid.minor = element_blank(),
    panel.border = element_blank(),
    panel.background = element_blank()
  ) + 
  scale_color_manual(values = c("Older" = '#bf812d', "Young" = '#35978f')) +
  geom_smooth(method = "lm", se = TRUE, linewidth = 0.3) +
  facet_wrap2(~AIFI_L3, axes = "all", remove_labels = "all", ncol = 1, scale = 'free_y') +
  theme(
    panel.grid.major = element_blank(),
    panel.grid.minor = element_blank(),
    strip.background = element_rect(fill = adjustcolor(color_vector[selected_celltype], alpha.f = 0.8), color = NA),
    strip.text = element_text(color = is_dark_color(adjustcolor(color_vector[selected_celltype], alpha.f = 0.8)))
  ) +
  xlab('') +
  theme(legend.position = "none")

p2 <- ggplot(df_sub, aes(AIFI_L3_clr, fill = `Age Group`, color = `Age Group`)) +
  geom_density(alpha = 0.5, linewidth = 0.5) +
  scale_fill_manual(values = c("Older" = '#bf812d', "Young" = '#35978f')) +
  scale_color_manual(values = c("Older" = '#bf812d', "Young" = '#35978f')) +
  coord_flip() +
  theme_void(base_size = 6) +
  theme(legend.position = "none") +
  theme(plot.margin = margin(0, 0, 0, -1, "mm"))

combined_plot <- plot_grid(p1, p2, ncol = 2, rel_widths = c(10, 0.4), align = 'h', axis = 'tb')
plot_list[[selected_celltype]]<-combined_plot
}


ERROR: [1m[33mError[39m in `combine_vars()`:[22m
[1m[22m[33m![39m Faceting variables must have at least one value.


In [36]:
options(repr.plot.width = 120/25.4, repr.plot.height = 25/25.4)

p<-plot_grid(plot_list[[1]],plot_list[[2]], 
             ncol = 2, rel_widths = c(10, 10), 
             align = 'h', axis = 'tb')
p
ggsave('FigureB.pdf',p,width = 120/25.4,height = 25/25.4)

ERROR: Error in plot_list[[1]]: subscript out of bounds
